<a href="https://colab.research.google.com/github/vrunda-28/CS6910_2021_Vrunda_Mudit/blob/main/Dl_assignment_v5(with_wandb).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
np.random.seed(10)
from matplotlib import pyplot
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback
#data download


In [ ]:
wandb.login()

True

In [ ]:
def train():

  default_configs = {"epochs": 3 ,
                     "learning_rate": 0.01,
                     "No_hidden_layer": 3,
                     "Neurons": 32,
                     "weight_decay": 0,
                     "mini_batch_size": 64,
                     "weight_initialization": 'random',
                     "activation_type": "tanh",
                     "loss_optimizer_type": "adam",
                     }
  
  run = wandb.init(project='CS6910_assignment1', config=default_configs)
  config = wandb.config

  class nn_from_scratch:

    X_train = []
    Y_train = []
    X_validation = []
    Y_validation = []
    X_test = []

    in_dim = 784
    out_dim = 10
    W = []
    W_zeros = []
    delta_W_memory = []
  
    previous_update = []
    previous_m = []
    previous_v = []
    vt_hat_sqrt=[]
    v_t_sqrt = []      #use this in rmsprop
    eps = 1e-8    
    beta_1 = 0.9
    beta_2 = 0.999
    beta = 0.9
    t = 0
    
    
    def __init__(self):

      self.No_hidden_layer = config.No_hidden_layer
      self.Neurons = config.Neurons
      self.batch_size = config.mini_batch_size
      self.epoch_num = config.epochs
      self.init_type = config.weight_initialization
      self.activation_type = config.activation_type
      self.loss_type = 'cross_entropy'
      self.gradient_type = config.loss_optimizer_type

      self.weight_decay = config.weight_decay
      self.activation = []
      self.pre_activation = []
      self.eta = config.learning_rate
      self.gamma = 0.9
      self.No_of_Neurons = []

      for i in range(self.No_hidden_layer):
        self.No_of_Neurons.append(self.Neurons)



  ################################################################################    
      
      #Initializing weights with given type
      if self.init_type == 'random':

        start = self.in_dim
        for i in range(self.No_hidden_layer):
          j = self.No_of_Neurons[i]
          self.W.append(np.random.normal(scale=0.1, size=(j,start+1)))
          start = j
        self.W.append(np.random.normal(scale = 0.1, size=(self.out_dim,j+1)))

      elif self.init_type == 'xavier':
        
        start = self.in_dim
        for i in range(self.No_hidden_layer):
          j = self.No_of_Neurons[i]
          limit = np.sqrt(6 / (j + start+1))
          self.W.append(np.random.uniform(low = -limit, high = limit, size=(j, start+1)))
          start = j
        limit = np.sqrt(6 / (self.out_dim + j+1))
        self.W.append(np.random.uniform(low = -limit, high = limit, size=(self.out_dim, j+1)))
      
      else:
        print('Initialisation type is not in list')

  ################################################################################
      
      # Initializing all temporary parameters to zero
      start = self.in_dim
      for i in range(self.No_hidden_layer):
        j = self.No_of_Neurons[i]
        self.W_zeros.append(np.zeros((j,start+1)))
        start = j
      self.W_zeros.append(np.zeros((self.out_dim,j+1)))


      self.previous_update = np.copy(self.W_zeros)
      self.previous_m = np.copy(self.W_zeros)
      self.previous_v = np.copy(self.W_zeros)       

  ################################################################################
  
    #Forward_propogation algorithm returning output probability vector
    def FFN_return_output(self, input):
      h = (input.T).copy()     
      self.activation = []        
      self.pre_activation = []

      for i in range(self.No_hidden_layer):
        a = np.dot(self.W[i],np.vstack((h,1)))
        self.pre_activation.append(a)
        h = self.activation_function(a)
        self.activation.append(h)
        h = self.activation[-1]
        del a

      a = np.dot(self.W[-1],np.vstack((h,1)))
      self.pre_activation.append(a)
      self.activation.append(self.softmax(a))
      return self.activation[-1]

  ################################################################################
    
    # Computing labels for given input
    def predict_model(self,input):
      label_pred = []
      for i in range(input.shape[0]):
        Y_pred = self.FFN_return_output(np.array([input[i]]))
        label_pred.append(np.argmax(Y_pred))
      return label_pred

  ################################################################################

    # Computing Loss          
    def loss_calculation(self, input,ground_truth):

      loss = []
      label = []
      for i in range(input.shape[0]) :
        Y_pred = self.FFN_return_output(np.array([input[i]]))
        label.append(np.argmax(Y_pred))
        if self.loss_type == 'cross_entropy':
          loss.append(np.multiply(np.log(np.add(Y_pred.T,self.eps)).dot(np.array([ground_truth[i]]).T), (-1)))                     
        elif self.loss_type == 'square_error':
          loss.append(np.sum((Y_pred-ground_truth[i].T)**2)*.5)
        else:
          print("Wrong loss function entered")
      return np.mean(loss)

  ################################################################################

    # Update parameters in batches 
    def batch_update(self,index):

      delta_W_memory=np.copy(self.W_zeros)
      for i in range(index*self.batch_size,(index+1)*self.batch_size):  
        #tmp = self.gradient_descent(np.array([self.X_train[i]]),np.array([self.Y_train[i]]))
        tmp = np.multiply(self.gradient_descent(np.array([self.X_train[i]]),np.array([self.Y_train[i]])), (1/self.batch_size))
        delta_W_memory = np.add(tmp,delta_W_memory)      
        del tmp
      W_batch_update = self.loss_optimiser(delta_W_memory, self.X_train, self.Y_train, index)
      # temp_W = self.W
      self.W = W_batch_update - np.multiply(self.eta*self.weight_decay, self.W) #L2 regularisation
      #self.W = W_batch_update
      del W_batch_update

  ################################################################################
    # Back propogation algorithm returning gradients
    def gradient_descent(self,input,ground_truth) :
      self.FFN_return_output(input)
      
      delta_a = []
      delta_W = []
      if self.loss_type == 'square_error':
        print("Mean-squared loss activated")
      elif self.loss_type == 'cross_entropy':
        delta_a.append(self.activation[-1]-ground_truth.T)
      else:
        print("Wrong loss function entered")

      for i in reversed(range(self.No_hidden_layer)):
        delta_h = self.W[i+1].T.dot(delta_a[-1])
        delta_a.append(delta_h[:-1,:] * self.activation_function_der(self.pre_activation[i]))
        del delta_h
      for index in range(self.No_hidden_layer+1):
        rev_index = self.No_hidden_layer-index
        if index == 0:
          output_layer = np.vstack((input.T,1))
        else:
          output_layer = np.vstack((self.activation[index -1],1))
        delta_W.append(delta_a[rev_index].dot(output_layer.T))
      return delta_W

  ################################################################################
    
    #Types of Loss optimiser functions
    def loss_optimiser(self, delta_W_memory, input, ground_truth, index):
      if self.gradient_type == 'vanila':
        W_updated = self.do_vanila(delta_W_memory, input, ground_truth, index)

      elif self.gradient_type == 'momentum':
        W_updated = self.do_momentum(delta_W_memory, input, ground_truth, index)

      elif self.gradient_type == 'nag':
        W_updated = self.do_nag(delta_W_memory, input, ground_truth, index)

      elif self.gradient_type == 'rmsprop':
        W_updated = self.do_rmsprop(delta_W_memory, input, ground_truth, index)

      elif self.gradient_type == 'adam':
        W_updated = self.do_adam(delta_W_memory, input, ground_truth, index)

      elif self.gradient_type == 'nadam':
        W_updated = self.do_nadam(delta_W_memory, input, ground_truth, index)

      else:
        print('Loss optimiser type is not in the list')
      return W_updated

  ################################################################################
    
    #Computing vanila gradient descent
    def do_vanila(self, delta_W_memory, input, ground_truth, index):
      W_updated = self.W - np.multiply(self.eta , delta_W_memory)
      return W_updated

  ################################################################################
    
    #Computing momentum gradient descent
    def do_momentum(self, delta_W_memory, input, ground_truth, index):
      update_t = np.multiply(self.gamma, self.previous_update) + np.multiply(self.eta, delta_W_memory)#equation 1
      W_updated = self.W - update_t #equation 2
      self.previous_update = np.copy(update_t)
      return W_updated

  ################################################################################
    
    #Computing nag gradient descent
    def do_nag(self, delta_W_memory, input, ground_truth, index):
      delta_W_look_ahead = self.W_zeros  
      W_look_ahead = self.W - np.multiply(self.gamma , self.previous_update) #equation 1

      W_temp1 = np.copy(self.W) #Temporary store as self.W is acquired gy gd
      self.W = np.copy(W_look_ahead) 

      for i in range(index*self.batch_size,(index+1)*self.batch_size):
        tmp= self.gradient_descent(np.array([self.X_train[i]]),np.array([self.Y_train[i]]))
        delta_W_look_ahead = np.add(tmp,delta_W_look_ahead)
        del tmp
      
      self.W = np.copy(W_temp1) # Temporary restoring back

      update_t = np.multiply(self.gamma, self.previous_update) + np.multiply(self.eta, delta_W_look_ahead) #equation 2
      W_updated = self.W - update_t #equation 3
      self.previous_update = np.copy(update_t)
      return W_updated

  ################################################################################
    
    #Computing rmsprop gradient descent
    def do_rmsprop(self, delta_W_memory, input, ground_truth, index):
      
      self.v_t_sqrt = np.copy(self.W_zeros)
      delta_W_square = np.multiply(delta_W_memory, delta_W_memory)
      v_t = np.multiply(self.beta, self.previous_v) + np.multiply(1-self.beta, delta_W_square)#equation 1

      for i in range(len(v_t)):
        self.v_t_sqrt[i] = (self.eta/np.sqrt(v_t[i])+self.eps)

      W_updated = self.W - np.multiply(self.v_t_sqrt,delta_W_memory)#equation 2
      self.previous_v = np.copy(v_t)
      return W_updated

  ################################################################################
    
    #Computing adam gradient descent
    def do_adam(self, delta_W_memory, input, ground_truth, index):
      
      self.vt_hat_sqrt=np.copy(self.W_zeros)
      m_t = np.multiply(self.beta_1 , self.previous_m) + np.multiply((1-self.beta_1) , delta_W_memory)
      v_t = np.multiply(self.beta_2 , self.previous_v) + np.multiply( (1-self.beta_2), np.multiply(delta_W_memory,delta_W_memory))

      mt_hat = m_t/(1-np.power(self.beta_1, (self.t+1)))   
      vt_hat = v_t/(1-np.power(self.beta_2, (self.t+1)))

      for i in range(len(vt_hat)):                  
          self.vt_hat_sqrt[i]=self.eta/(np.sqrt(vt_hat[i]+self.eps))

      W_updated = self.W - np.multiply(self.vt_hat_sqrt, mt_hat)
      self.previous_m = np.copy(m_t)
      self.previous_v = np.copy(v_t)
      return W_updated

  ################################################################################
    
    #Computing nadam gradient descent
    def do_nadam(self, delta_W_memory, input, ground_truth, index):

      self.vt_hat_sqrt=np.copy(self.W_zeros)
      m_t = np.multiply(self.beta_1 , self.previous_m) + np.multiply((1-self.beta_1) , delta_W_memory)
      v_t = np.multiply(self.beta_2 , self.previous_v) + np.multiply( (1-self.beta_2), np.multiply(delta_W_memory,delta_W_memory))

      mt_hat = m_t/(1-np.power(self.beta_1, (self.t+1)))   
      vt_hat = v_t/(1-np.power(self.beta_2, (self.t+1)))

      for i in range(len(vt_hat)):                  
          self.vt_hat_sqrt[i]=self.eta/(np.sqrt(vt_hat[i]+self.eps))

      beta_factor = (1-self.beta_1)/(1-np.power(self.beta_1, (self.t+1)))
      mul_factor = np.multiply(self.beta_1, mt_hat) + np.multiply(beta_factor, delta_W_memory)

      W_updated = self.W - np.multiply(self.vt_hat_sqrt, mul_factor)
      self.previous_m = np.copy(m_t)
      self.previous_v = np.copy(v_t)
      return W_updated

  ################################################################################
    
    # Types of activation function
    def activation_function(self,input):

      if self.activation_type == 'sigmoid':
        return (1/(1+np.exp(-input)))
      elif self.activation_type == 'tanh':
        return (np.tanh(input))
      elif self.activation_type =='softmax':
        return np.exp(input)/np.sum(np.exp(input))
      elif self.activation_type =='ReLU':
        return np.maximum(0, input)
      else:
        print('Activation Function not in list')

  ###############################################################################
    
    #Computing derivative of given activation function
    def activation_function_der(self,input):

      if self.activation_type == 'sigmoid': 
        temp = 1/(1+np.exp(-input))
        return (temp*(1-temp))
      elif self.activation_type == 'tanh':
        return (1-np.multiply(np.tanh(input), np.tanh(input)))
      elif self.activation_type == 'ReLU':
        temp = np.sign(input)
        return np.maximum(0, temp)
      else:
        print('Derivative Function not in list')

  ###############################################################################

    # Softmax
    def softmax(self,x):
    
      x = np.exp(x)/np.sum(np.exp(x))
      x[np.isnan(x)] = 1
      return x
  
  ###############################################################################
   
    #Calculating loss and accuracy for traiing and validation set 
    def Loss_Accuracy_calculation(train_true_labels,val_true_labels):

      train_loss = FFN.loss_calculation(FFN.X_train, FFN.Y_train)
      train_pred_labels = FFN.predict_model(FFN.X_train)
      validation_loss = FFN.loss_calculation(FFN.X_validation,FFN.Y_validation)
      val_pred_labels = FFN.predict_model(FFN.X_validation)
      training_accuracy = sum([1 for i,j in zip(train_pred_labels,train_true_labels) if i==j])/len(train_pred_labels)
      validation_accuracy = sum([1 for i,j in zip(val_pred_labels,val_true_labels) if i==j])/len(val_pred_labels)

      print('   Train_loss==> ',train_loss)
      print('   Val_loss====> ',validation_loss)
      print('')
      print('   Train_acc==> ',training_accuracy)
      print('   Val_acc=====> ',validation_accuracy)

  #load dataset
  (X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
  class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                  'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


  # Reshaping dataset
  X_train = X_train.reshape(len(X_train),784)
  X_test = X_test.reshape(len(X_test),784)

  # Creating object
  FFN = nn_from_scratch()

  # Normalizing Dataset

  X_train = (X_train - np.mean(X_train,axis=0))/(np.std(X_train,axis=0) + FFN.eps)
  X_test = (X_test - np.mean(X_test,axis=0))/(np.std(X_test,axis=0) + FFN.eps)


  #splitting train and validation data
  X_train, X_validation,Y_train,Y_validation = train_test_split(X_train,Y_train, test_size=0.1, shuffle=True)
  train_true_labels = Y_train
  val_true_labels = Y_validation

  FFN.X_train = X_train
  FFN.X_test = X_test
  FFN.X_validation = X_validation

  #onehot encoding
  FFN.Y_train = np.eye(FFN.out_dim)[Y_train]
  FFN.Y_validation = np.eye(FFN.out_dim)[Y_validation]


  ###############################################################################

  def Loss_Accuracy_calculation(train_true_labels,val_true_labels):
    train_loss = FFN.loss_calculation(FFN.X_train, FFN.Y_train)
    train_pred_labels = FFN.predict_model(FFN.X_train)
    validation_loss = FFN.loss_calculation(FFN.X_validation,FFN.Y_validation)
    val_pred_labels = FFN.predict_model(FFN.X_validation)
    training_accuracy = (sum([1 for i,j in zip(train_pred_labels,train_true_labels) if i==j])/len(train_pred_labels))*100
    validation_accuracy = (sum([1 for i,j in zip(val_pred_labels,val_true_labels) if i==j])/len(val_pred_labels))*100

    print('   Train_loss==> ',train_loss)
    print('   Val_loss====> ',validation_loss)
    print('')
    print('   Train_acc==> ',training_accuracy)
    print('   Val_acc=====> ',validation_accuracy)

    test_label = FFN.predict_model(FFN.X_test)      

    accuracy = (sum([1 for i,j in zip(test_label,Y_test) if i==j])/len(Y_test))*100
    print('Final_accuracy==>  ',accuracy)

    return train_loss,validation_loss,training_accuracy,validation_accuracy,accuracy,test_label

####################################################################################
  
  print('epoch===>',0)
  train_loss,validation_loss,training_accuracy,validation_accuracy,accuracy,test_label = Loss_Accuracy_calculation(train_true_labels,val_true_labels)
  wandb.log({"epoch" : 0,
          "accuracy": accuracy,
          "val_accuracy" : validation_accuracy,
          "val_loss": validation_loss,
          "train_loss": train_loss,
          "train_accuracy":training_accuracy})

  for epoch in range(1,FFN.epoch_num+1):
    delta_W_memory = np.copy(FFN.W_zeros)
    time_step = 0
    print('Epoch==>',epoch)

    No_of_batches = (FFN.X_train).shape[0]//FFN.batch_size
    for i in range(No_of_batches):
      FFN.batch_update(i)
      FFN.t += 1
      time_step += 1
    train_loss,validation_loss,training_accuracy,validation_accuracy,accuracy,test_label = Loss_Accuracy_calculation(train_true_labels,val_true_labels)  
    wandb.log({"epoch" : epoch,
              "accuracy": accuracy,
              "val_accuracy" : validation_accuracy,
              "val_loss": validation_loss,
              "train_loss": train_loss,
              "train_accuracy":training_accuracy})
                
  wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                        y_true=Y_test, preds=test_label,
                        class_names=class_names)})

  ################################################################################

  

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
        'No_hidden_layer': {
            'values': [3,4,5]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },'Neurons':{
            'values': [32,64,128]
        },
        'weight_decay': {
            'values': [0,0.0005,0.5]
        },'mini_batch_size':{
            'values': [1,16,32,64]
        },'loss_optimizer_type':{
            'values': ['vanila','momentum','nag','rmsprop','adam','nadam']
        },'weight_initialization': {
            'values': ['random','xavier']
        },'activation_type':{
            'values': ['sigmoid','tanh','ReLU']
        },

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910_Assignment1")

Create sweep with ID: 1o5dqarx
Sweep URL: https://wandb.ai/vrunda/CS6910_Assignment1/sweeps/1o5dqarx


In [ ]:
wandb.agent(sweep_id, train,count = 100)

wandb: Agent Starting Run: 3gfxz89q with config:
wandb: 	Neurons: 32
wandb: 	No_hidden_layer: 5
wandb: 	activation_type: ReLU
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0001
wandb: 	loss_optimizer_type: nadam
wandb: 	mini_batch_size: 64
wandb: 	weight_decay: 0.0005
wandb: 	weight_initialization: random
wandb: WARNING Ignored wandb.init() arg project when running a sweep


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:792: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, order=order, subok=subok, copy=True)


epoch===> 0
   Train_loss==>  2.313226759087145
   Val_loss====>  2.3126950468318

   Train_acc==>  11.103703703703703
   Val_acc=====>  11.333333333333332
Final_accuracy==>   10.71
Epoch==> 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:318: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


   Train_loss==>  0.6818197837931285
   Val_loss====>  0.6837114258656299

   Train_acc==>  74.09259259259258
   Val_acc=====>  73.85000000000001
Final_accuracy==>   73.86
Epoch==> 2
   Train_loss==>  0.5400097473847606
   Val_loss====>  0.5408218170542948

   Train_acc==>  78.9388888888889
   Val_acc=====>  79.03333333333333
Final_accuracy==>   78.08
Epoch==> 3
   Train_loss==>  0.47926450171829194
   Val_loss====>  0.48046535608714147

   Train_acc==>  82.30740740740741
   Val_acc=====>  82.33333333333334
Final_accuracy==>   81.13
Epoch==> 4
   Train_loss==>  0.44580247079207425
   Val_loss====>  0.4498394901594803

   Train_acc==>  83.84814814814816
   Val_acc=====>  83.91666666666666
Final_accuracy==>   82.27
Epoch==> 5
   Train_loss==>  0.4239401200833761
   Val_loss====>  0.43178816572411105

   Train_acc==>  84.68148148148148
   Val_acc=====>  84.53333333333333
Final_accuracy==>   83.02000000000001


...

epoch===> 0
   Train_loss==>  2.458779224853956
   Val_loss====>  2.4485701696280247

   Train_acc==>  9.974074074074075
   Val_acc=====>  10.233333333333333
Final_accuracy==>   10.0
Epoch==> 1
   Train_loss==>  0.4405887768753292
   Val_loss====>  0.46893434946856016

   Train_acc==>  84.14814814814815
   Val_acc=====>  83.21666666666667
Final_accuracy==>   82.17
Epoch==> 2
   Train_loss==>  0.36355132773762266
   Val_loss====>  0.40828471487363605

   Train_acc==>  86.70925925925927
   Val_acc=====>  85.63333333333333
Final_accuracy==>   84.28999999999999
Epoch==> 3
   Train_loss==>  0.32989434795545325
   Val_loss====>  0.3910857477815864

   Train_acc==>  87.81851851851852
   Val_acc=====>  86.13333333333333
Final_accuracy==>   84.92
Epoch==> 4
   Train_loss==>  0.3131942179475104
   Val_loss====>  0.3893305418758393

   Train_acc==>  88.17777777777778
   Val_acc=====>  86.08333333333333
Final_accuracy==>   84.7
Epoch==> 5
   Train_loss==>  0.2989774859405348
   Val_loss====>  0.38

epoch,5
accuracy,84.78
val_accuracy,86.0
val_loss,0.38576
train_loss,0.29898
train_accuracy,88.57222
_runtime,571
_timestamp,1615793316
_step,6


epoch,▁▂▄▅▇█
accuracy,▁█████
val_accuracy,▁█████
val_loss,█▁▁▁▁▁
train_loss,█▁▁▁▁▁
train_accuracy,▁█████
_runtime,▁▂▄▅▇██
_timestamp,▁▂▄▅▇██
_step,▁▂▃▅▆▇█


wandb: Agent Starting Run: 788pih63 with config:
wandb: 	Neurons: 32
wandb: 	No_hidden_layer: 5
wandb: 	activation_type: tanh
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	loss_optimizer_type: nag
wandb: 	mini_batch_size: 32
wandb: 	weight_decay: 0
wandb: 	weight_initialization: random
wandb: WARNING Ignored wandb.init() arg project when running a sweep


epoch===> 0
   Train_loss==>  2.3198098951773685
   Val_loss====>  2.3189351825141995

   Train_acc==>  6.064814814814815
   Val_acc=====>  6.2
Final_accuracy==>   6.03
Epoch==> 1
   Train_loss==>  0.5657722200249102
   Val_loss====>  0.5822141698120177

   Train_acc==>  79.82222222222222
   Val_acc=====>  79.08333333333334
Final_accuracy==>   78.7
Epoch==> 2
   Train_loss==>  0.4451140187187288
   Val_loss====>  0.47912078641235994

   Train_acc==>  84.39629629629628
   Val_acc=====>  83.63333333333334
Final_accuracy==>   82.53
Epoch==> 3
   Train_loss==>  0.3981684132166816
   Val_loss====>  0.4495588004598314

   Train_acc==>  85.98148148148148
   Val_acc=====>  84.89999999999999
Final_accuracy==>   83.50999999999999
Epoch==> 4
